In [14]:
import jqdata               #导入聚宽函数库
from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import talib
from __future__ import division

import matplotlib.pyplot as plt


stock_to_choose = get_fundamentals(query(
        valuation.code, valuation.pe_ratio, 
        valuation.pb_ratio,valuation.market_cap, 
        indicator.eps, indicator.inc_net_profit_annual
    ).filter(
#         indicator.inc_net_profit_annual > 50
#         valuation.pe_ratio < 100,
        valuation.pe_ratio > 2
    ).order_by(
        valuation.market_cap.asc()
    ), date=None)

stock_list_all = list(stock_to_choose['code'])

# stock_list_all = get_index_stocks('000300.XSHG')


print stock_list_all

['300029.XSHE', '300321.XSHE', '300354.XSHE', '300489.XSHE', '600696.XSHG', '002205.XSHE', '002633.XSHE', '300405.XSHE', '600768.XSHG', '300069.XSHE', '600605.XSHG', '002715.XSHE', '002667.XSHE', '600766.XSHG', '002816.XSHE', '002209.XSHE', '603031.XSHG', '600455.XSHG', '300092.XSHE', '600561.XSHG', '300417.XSHE', '300330.XSHE', '300551.XSHE', '300093.XSHE', '600137.XSHG', '300275.XSHE', '600095.XSHG', '300573.XSHE', '603090.XSHG', '603088.XSHG', '002857.XSHE', '300592.XSHE', '000702.XSHE', '603029.XSHG', '000004.XSHE', '002058.XSHE', '600634.XSHG', '002778.XSHE', '300665.XSHE', '600275.XSHG', '002862.XSHE', '300442.XSHE', '300211.XSHE', '600556.XSHG', '002134.XSHE', '603580.XSHG', '300387.XSHE', '600272.XSHG', '300521.XSHE', '300464.XSHE', '300483.XSHE', '600250.XSHG', '600444.XSHG', '600379.XSHG', '600615.XSHG', '600573.XSHG', '300499.XSHE', '000668.XSHE', '603536.XSHG', '000679.XSHE', '000713.XSHE', '300557.XSHE', '300305.XSHE', '002105.XSHE', '002729.XSHE', '600235.XSHG', '300460.X

选取股票


In [15]:
stock_list_all = stock_list_all[:]
# stock_list_all = ['002558.XSHE', '600362.XSHG', '600516.XSHG', '603260.XSHG', '000898.XSHE', '002202.XSHE', '600271.XSHG', '600066.XSHG', '600377.XSHG', '002602.XSHE']
# stock_list_all = ['002558.XSHE']
print stock_list_all

['300029.XSHE', '300321.XSHE', '300354.XSHE', '300489.XSHE', '600696.XSHG', '002205.XSHE', '002633.XSHE', '300405.XSHE', '600768.XSHG', '300069.XSHE', '600605.XSHG', '002715.XSHE', '002667.XSHE', '600766.XSHG', '002816.XSHE', '002209.XSHE', '603031.XSHG', '600455.XSHG', '300092.XSHE', '600561.XSHG', '300417.XSHE', '300330.XSHE', '300551.XSHE', '300093.XSHE', '600137.XSHG', '300275.XSHE', '600095.XSHG', '300573.XSHE', '603090.XSHG', '603088.XSHG', '002857.XSHE', '300592.XSHE', '000702.XSHE', '603029.XSHG', '000004.XSHE', '002058.XSHE', '600634.XSHG', '002778.XSHE', '300665.XSHE', '600275.XSHG', '002862.XSHE', '300442.XSHE', '300211.XSHE', '600556.XSHG', '002134.XSHE', '603580.XSHG', '300387.XSHE', '600272.XSHG', '300521.XSHE', '300464.XSHE', '300483.XSHE', '600250.XSHG', '600444.XSHG', '600379.XSHG', '600615.XSHG', '600573.XSHG', '300499.XSHE', '000668.XSHE', '603536.XSHG', '000679.XSHE', '000713.XSHE', '300557.XSHE', '300305.XSHE', '002105.XSHE', '002729.XSHE', '600235.XSHG', '300460.X

适当调整筛选

In [16]:

stat = {}
print stat
for stock_name in stock_list_all:
    stock_price = get_price(stock_name, count = 500, end_date='2018-5-31', frequency='daily', 
                            fields=['open', 'close'],skip_paused=True, fq='none')

    stock_price[stock_price['close']==0]=np.nan
    close_price= stock_price.dropna()

    close_price = stock_price['close']
    

    
    if len(close_price) > 20:
        dif, dea, macd = talib.MACD(stock_price['close'].values, fastperiod=65, slowperiod=130, signalperiod=45)

        id_x_gold = np.where((macd[:-1] < 0) & (macd[1:] > 0))[0] + 1

        id_x_gold = [elem for elem in id_x_gold if elem < len(close_price) - 10 ]  # 防止越界

 
        
        total_count = 0
        win_count = [0 for i in range(10)]    #用数组存储胜利的次数，从两天后开始计算
        lose_count = [0 for i in range(10)]   #用数据存储失败的次数，从两天后开始计算

        if len(id_x_gold) > 0:
            for position in id_x_gold:
                if dif[position] > 0:
                    total_count += 1

                    if close_price[position] < close_price[position + 2]:
                        win_count[0] += 1
                    else:
                        lose_count[0] += 1

                    if close_price[position] < close_price[position + 3]:
                        win_count[1] += 1
                    else:
                        lose_count[1] += 1

                    if close_price[position] < close_price[position + 4]:
                        win_count[2] += 1
                    else:
                        lose_count[2] += 1

                    if close_price[position] < close_price[position + 5]:
                        win_count[3] += 1
                    else:
                        lose_count[3] += 1

                    if close_price[position] < close_price[position + 6]:
                        win_count[4] += 1
                    else:
                        lose_count[4] += 1

                    if close_price[position] < close_price[position + 7]:
                        win_count[5] += 1
                    else:
                        lose_count[5] += 1

                    if close_price[position] < close_price[position + 8]:
                        win_count[6] += 1
                    else:
                        lose_count[6] += 1
            
                    if close_price[position] < close_price[position + 9]:
                        win_count[7] += 1
                    else:
                        lose_count[7] += 1
                        
                    if close_price[position] < close_price[position + 10]:
                        win_count[8] += 1
                    else:
                        lose_count[8] += 1    
    
                    if close_price[position] < close_price[position + 11]:
                        win_count[9] += 1
                    else:
                        lose_count[9] += 1
    
    
    
    stat[stock_name] = [total_count,win_count,lose_count ]

print stat

{}


{'002897.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '002233.XSHE': [1, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], '002906.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '002596.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '600387.XSHG': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '000836.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '000988.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '300643.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '002060.XSHE': [1, [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 1, 1, 1]], '300033.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '300485.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], '002858.XSHE': [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0,

开始针对搜集的统计数据进行分析


In [17]:
totals = 0
wins = [0 for i in range(10)]
loses = [0 for i in range(10)]
stock_win_list = []
for key in stat.keys():
    totals = totals + stat[key][0]
    wins = np.array(wins) + np.array(stat[key][1])
    loses = np.array(loses) + np.array(stat[key][2])

    
print totals,np.array(wins)/totals


1383 [ 0.44685466  0.4714389   0.46420824  0.46276211  0.46637744  0.46782357
  0.47650036  0.47794649  0.47433116  0.4736081 ]


分类看每个股票是否与整体的趋势